In [37]:
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.user import User
import pandas as pd
import re
import datetime
import time

In [38]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [39]:
FB_ACCESS_TOKEN = os.getenv("FB_ACCESS_TOKEN")
FB_APP_SECRET = os.getenv("FB_APP_SECRET")
FB_APP_ID = os.getenv("FB_APP_ID")
FB_USER_ID = os.getenv("FB_USER_ID")

FacebookAdsApi.init(access_token=FB_ACCESS_TOKEN, app_secret=FB_APP_SECRET, app_id=FB_APP_ID)

In [40]:
# Obtém o usuário autenticado
user = User(FB_USER_ID)

# Lista as contas de anúncios com os campos especificados
ad_accounts = user.get_ad_accounts(fields=['id', 'name'])

# Exibe as contas de anúncios
for account in ad_accounts:
    print(f"ID: {account.get('id', 'N/A')}, Nome: {account.get('name', 'Sem Nome')}")

ID: act_447307204124409, Nome: CA - Marinna Flores (R)
ID: act_448334200520743, Nome: CA - Gabriel Jardim
ID: act_167305115741886, Nome: META-JM3D
ID: act_611842051044792, Nome: CA - Oficina Coworking
ID: act_2025557001129719, Nome: CA - Dr Elisa
ID: act_642540421310400, Nome: CA- Casa dos Vidros
ID: act_915548350011111, Nome: CA - Bonna Pizza
ID: act_1463395761238411, Nome: More Drinks
ID: act_1206262810375779, Nome: Felipe França
ID: act_346946485131658, Nome: CA - Wolf Soluções (P)
ID: act_1603605473705233, Nome: CA - Wolf Soluções (Head)


In [41]:
AD_ACCOUNT_LIST = [
    'act_915548350011111',  # Bonna Pizza
]

### Extração das campanhas

In [42]:
def _etl_general(dataset):  # Method used in fb_entidades, to make a first etl proccess.

    for each in dataset:
        each['created_time'] = each['created_time'][:10]

    for record in dataset:
        if 'daily_budget' in record:
            record['daily_budget'] = float(int(record['daily_budget']) / 100)
        if 'lifetime_budget' in record:
            record['lifetime_budget'] = float(int(record['lifetime_budget']) / 100)
        if 'budget_remaining' in record:
            record['budget_remaining'] = float(int(record['budget_remaining']) / 100)

    return dataset


def obter_campanhas():
    dataset_campaigns = list()  # Function Return
    fields = ['id', 'name', 'created_time', 'status', 'account_id', 'daily_budget', 'lifetime_budget',
              'objective']
    params = {'limit': '5000'}

    for acc in AD_ACCOUNT_LIST:
        call = AdAccount(acc).get_campaigns(fields=fields, params=params)

        # Converte cada objeto "<Campaign>" em Dict e insere como um elemento da lista.
        for campanha in call:
            dataset_campaigns.append(dict(campanha))

    return _etl_general(dataset_campaigns)


def etl_campaign_name(dataset):  # -> Receive a list Object.
    df = pd.DataFrame(dataset)
    regex = r'\[[\w\s\.\-\%\+]+\]'

    # Create a custom column with the kind of campaign (New or Old).
    # df.insert(loc=4, column='campaign_group', value=np.where(df.created_time <= '2022-06-27',
    #                                                          'padrao_antigo', 'padrao_novo'))

    df['campaign_group'] = ["padrao_antigo" if camp <= '2022-06-27' else "padrao_novo" for camp in df.created_time]

    # Creating a new column with lower values.
    df.insert(loc=2, column='name_etl', value=df.name.str.lower())

    # Excluding accents from name_etl column.
    df.name_etl = df.name_etl.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

    # Conditional to insert '[sem_autor]' before old campaigns names.
    df.loc[df['campaign_group'] == 'padrao_antigo', 'name_etl'] = '[sem_autor]' + df.name_etl.astype(str)

    df[['lifetime_budget', 'daily_budget']] = df[['lifetime_budget', 'daily_budget']].fillna(0)

    # Extracting the first pattern and inserting into a new column.
    # temp_lst = list()
    # for row in df.name_etl:
    #     match = re.findall(regex, row)
    #     temp_lst.append(match[0])
    # series = pd.Series(temp_lst)
    # df.insert(loc=3, column='pattern1', value=series)

    # # Extracting the second pattern and inserting into a new column.
    # temp_lst = list()
    # for row in df.name_etl:
    #     match = re.findall(regex, row)
    #     try:
    #         temp_lst.append(match[1])
    #     except IndexError:
    #         temp_lst.append('Nao especificado')
    # series = pd.Series(temp_lst)
    # df.insert(loc=4, column='pattern2', value=series)

    # # Extracting the third pattern and inserting into a new column.
    # temp_lst = list()
    # for row in df.name_etl:
    #     match = re.findall(regex, row)
    #     try:
    #         temp_lst.append(match[2])
    #     except IndexError:
    #         temp_lst.append('Nao especificado')
    # series = pd.Series(temp_lst)
    # df.insert(loc=4, column='pattern3', value=series)
    # df['pattern3'].replace(['[eduardo costa]', '[eduardo sertanejo]'], '[eduardo-costa]', inplace=True)

    return df

In [43]:
campaigns = obter_campanhas()
campaigns = etl_campaign_name(campaigns)

campaigns

,id,name,name_etl,created_time,status,account_id,objective,daily_budget,lifetime_budget,campaign_group
0,120215376828680137,WS - 12 [Venda] [Aberto] [Cardápio] [6km] - 26...,ws - 12 [venda] [aberto] [cardapio] [6km] - 26...,2025-02-26,ACTIVE,915548350011111,OUTCOME_SALES,0.0,0.0,padrao_novo
1,120211745920250137,D WS - 01 [Alcance] [BlackFiday] [CBO] 25/11/24,d ws - 01 [alcance] [blackfiday] [cbo] 25/11/24,2024-11-25,PAUSED,915548350011111,OUTCOME_AWARENESS,0.0,0.0,padrao_novo
2,120211144103230137,D WS 11 - [Venda] [Vespasiano] [Lista Repediu]...,d ws 11 - [venda] [vespasiano] [lista repediu]...,2024-10-29,PAUSED,915548350011111,OUTCOME_SALES,20.0,0.0,padrao_novo
3,120210647846290137,WS - 03 [ENGAJAMENTO] [IG] [CBO] - 09/10/24,ws - 03 [engajamento] [ig] [cbo] - 09/10/24,2024-10-09,ACTIVE,915548350011111,OUTCOME_ENGAGEMENT,20.0,0.0,padrao_novo
4,120210439810740137,WS - 10 [Venda] [Aberto] [Cardápio] [04km] [CB...,ws - 10 [venda] [aberto] [cardapio] [04km] [cb...,2024-10-02,PAUSED,915548350011111,OUTCOME_SALES,20.0,0.0,padrao_novo
5,120210242758180137,Publicação do Instagram: 20/09…...,publicacao do instagram: 20/09......,2024-09-24,PAUSED,915548350011111,LINK_CLICKS,10.0,0.0,padrao_novo
6,120210242711330137,"Publicação do Instagram: Na Bonna Pizza, cada...","publicacao do instagram: na bonna pizza, cada...",2024-09-24,PAUSED,915548350011111,LINK_CLICKS,10.0,0.0,padrao_novo
7,120210242698450137,Publicação do Instagram: VEM COM A MELHOR!❤️🍕...,publicacao do instagram: vem com a melhor!...,2024-09-24,PAUSED,915548350011111,LINK_CLICKS,10.0,0.0,padrao_novo
8,120209963372330137,D WS - 01 [Alcance] [Contratando] [CBO] 11/09/24,d ws - 01 [alcance] [contratando] [cbo] 11/09/24,2024-09-11,PAUSED,915548350011111,OUTCOME_AWARENESS,6.0,0.0,padrao_novo
9,120209543264750137,D WS - 01 [Engajamento] [WPP] [CBO] - 23/08/24,d ws - 01 [engajamento] [wpp] [cbo] - 23/08/24,2024-08-23,PAUSED,915548350011111,OUTCOME_SALES,15.0,0.0,padrao_novo


### Extração dos insights

In [44]:
# Data manual
time_range_param_manual = [datetime.date(2025, 3, 1), datetime.date(2025, 3, 5)]
date_diff = abs((time_range_param_manual[0] - time_range_param_manual[1]).days) + 1
date_lst = [time_range_param_manual[0] + datetime.timedelta(days=idx) for idx in range(date_diff)]

# Data dinamica
date_lst_today = list()
date_lst_today.append(datetime.date.today() - datetime.timedelta(1))

def obter_insights(adaccount):
    dataset_insights = list()
    fields = ['account_id', 'ad_id', 'spend', 'actions', 'clicks', 'impressions', 'reach', 'frequency']
    params = {
        'filtering': [{'field': "action_type", "operator": "CONTAIN", "value": "leadgen"}],
        'level': 'ad',
        'time_increment': '1',
        'limit': '1000',
        'breakdowns': ['device_platform'],
    }

    for date in date_lst:
        time.sleep(3)
        params['time_range'] = {'since': f'{date}', 'until': f'{date}'}
        call_to_api = AdAccount(adaccount).get_insights(fields=fields, params=params)  # Chamada da API

        for res in call_to_api:
            dataset_insights.append(dict(res))  # Converte objeto "<AdsInsights>"

        # info(f'{date} - Done')
    return dataset_insights


def etl_insights(insights):
    # for record in insights:
    #     if 'actions' in record:
    #         record['actions'] = int(record['actions'][0]['value'])

    insights_df = pd.DataFrame(insights)
    # insights_df['actions'].fillna(0, inplace=True)
    insights_df.drop('date_stop', axis=1, inplace=True)
    insights_df.rename(columns={'date_start': 'data_insights'}, inplace=True)

    dtypes = {
        'reach': int,
        'clicks': int,
        # 'actions': int,
        'impressions': int,
        'spend': float,
        'frequency': float
    }
    insights_df = insights_df.astype(dtypes)

    return insights_df

In [45]:
insights = obter_insights(AD_ACCOUNT_LIST[0])
insights = etl_insights(insights)
# inserir_insights('dispositivo', insights)

insights

,account_id,ad_id,spend,clicks,impressions,reach,frequency,data_insights,device_platform
0,915548350011111,120210647846360137,0.009315,0,1,1,1.000000,2025-03-01,desktop
1,915548350011111,120210647846360137,0.670685,2,273,210,1.300000,2025-03-01,mobile_app
2,915548350011111,120210647949810137,0.000000,1,2,2,1.000000,2025-03-01,desktop
3,915548350011111,120210647949810137,16.560012,12,4540,4301,1.055568,2025-03-01,mobile_app
4,915548350011111,120210647949810137,0.019988,0,6,6,1.000000,2025-03-01,mobile_web
...,...,...,...,...,...,...,...,...,...
213,915548350011111,120215589251220137,19.070000,23,4698,3115,1.508186,2025-03-05,mobile_app
214,915548350011111,120215589251220137,0.030000,1,9,8,1.125000,2025-03-05,mobile_web
215,915548350011111,120215626739070137,0.010000,0,4,4,1.000000,2025-03-05,desktop
216,915548350011111,120215626739070137,18.330000,22,3666,2670,1.373034,2025-03-05,mobile_app
